In [1]:
import sys, os, time
# %load_ext autoreload
# %autoreload 2
os.environ['FOUNDATION_RUN_MODE'] = 'jupyter'
os.environ['FOUNDATION_SAVE_DIR'] = '/is/ei/fleeb/workspace/chome/trained_nets'
os.environ['FOUNDATION_DATA_DIR'] = '/is/ei/fleeb/workspace/local_data'
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"
from IPython import display
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.distributions as distrib
import torch.multiprocessing as mp
import torchvision.models
import torchvision
from torch.utils.data import Dataset, DataLoader, TensorDataset
import gym
import numpy as np
%matplotlib notebook
# %matplotlib tk
import matplotlib.pyplot as plt
import imageio
import seaborn as sns
#plt.switch_backend('Qt5Agg') #('Qt5Agg')
import foundation as fd
from foundation import models
from foundation import util
from foundation import train
#from foundation.util import replicate, Cloner

from hybrid import get_model, get_data

np.set_printoptions(linewidth=120, suppress=True)

In [2]:
M = train.Run_Manager(tbout='/is/ei/fleeb/workspace/trained_nets/managed')
M.clear_links()
M.start_tb()

/is/ei/fleeb/workspace/trained_nets/managed is available to view runs on tensorboard


/is/ei/fleeb/workspace/foundation/foundation/train/config.py:58: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f)


Found 206 runs
Tensorboard started: http://localhost:6006/


In [5]:
#M.show()
M.clear_filters().select(model=None, dataset='3dshapes').filter(lambda r: not 'test' in r.name)
M.filter(lambda r: '200108' in r.name or '200109' in r.name or '200110' in r.name)
# M.filter(lambda r: '191218' in r.name)

# M.filter_idx(7, 5, 27, 23, 34, 16, 15, 13, 30, 28, 10, 12, 18, 21, 0, 1, 72, 73, 74, 67, 66, 57)


M.show()

  0 - 3dshapes-dvae_0100-5966131-07_200109-133026
  1 - 3dshapes-dvae_0100-5966131-08_200109-133040
  2 - 3dshapes-dvae_0101-5966132-01_200109-133045
  3 - 3dshapes-dvae_0101-5966132-02_200109-133053
  4 - 3dshapes-dvae_0102-5966133-07_200109-133118
  5 - 3dshapes-dvae_0102-5966133-08_200109-133107
  6 - 3dshapes-dvae_0122-5969012-01_200110-140610
  7 - 3dshapes-dvae_0122-5969012-02_200110-140616
  8 - 3dshapes-dwae_0090-5965922-04_200108-160125
  9 - 3dshapes-dwae_0092-5966021-04_200108-171824
 10 - 3dshapes-dwae_0094-5966044-04_200108-175514
 11 - 3dshapes-dwae_0095-5966045-04_200108-175850
 12 - 3dshapes-dwae_0096-5966047-04_200108-175913
 13 - 3dshapes-dwae_0097-5966051-04_200108-181233
 14 - 3dshapes-dwae_0100-5966131-05_200109-133027
 15 - 3dshapes-dwae_0100-5966131-06_200109-133124
 16 - 3dshapes-dwae_0101-5966132-00_200109-133044
 17 - 3dshapes-dwae_0102-5966133-05_200109-133202
 18 - 3dshapes-dwae_0102-5966133-06_200109-133106
 19 - 3dshapes-dwae_0122-5969012-00_200110-140613


In [5]:
ignore = None
ignore = ['disc_steps', '_type']
M.show_unique(ignore)

0) 3dshapes-vae_0077-5920872-21_191213-201539
	model.up_type - bilinear (nearest)

1) 3dshapes-vae_0077-5920872-22_191213-201525
	model.latent_reg_wt - 4 (1)
	model.up_type - bilinear (nearest)

2) 3dshapes-vae_0077-5920872-23_191213-201527
	model.latent_reg_wt - 0.25 (1)
	model.up_type - bilinear (nearest)

3) 3dshapes-vae_0088-5943512-00_191224-232237

4) 3dshapes-vae_0089-5943513-01_191224-232610
	model.latent_reg_wt - 4 (1)

5) 3dshapes-vae_0089-5943513-02_191224-232610
	model.latent_reg_wt - 8 (1)

6) 3dshapes-vae_0090-5965922-00_200108-160038
	model.generator.full_latent_dim - 16 (_)

7) 3dshapes-vae_0092-5966021-00_200108-171843
	model.generator.full_latent_dim - 16 (_)

8) 3dshapes-vae_0094-5966044-00_200108-175528

9) 3dshapes-vae_0095-5966045-00_200108-175855
	model.latent_dim - 8 (16)

10) 3dshapes-vae_0096-5966047-00_200108-175918
	model.latent_dim - 32 (16)

11) 3dshapes-vae_0097-5966051-00_200108-181230
	model.generator.full_latent_dim - 16 (_)

12) 3dshapes-vae_0100-5966

In [9]:
# M.link('{idx}__{model}__{unique}')
M.link('{date}__{model}__{unique}')

/is/ei/fleeb/workspace/foundation/foundation/train/config.py:58: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f)
Unable to get first event timestamp for run 200108-175855__vae__latent_dim=8: No event timestamp could be found
Unable to get first event timestamp for run 200108-175918__vae__latent_dim=32: No event timestamp could be found


In [6]:
# M.link('{name}__{unique}')

In [6]:
# os.listdir(M.tbout)

In [9]:
M.clear_links()

In [8]:
M.tbout

'/is/ei/fleeb/workspace/trained_nets/managed'

In [9]:
M.active[0].keys()

odict_keys(['name', 'path', 'ckpt_path', 'config', 'base', 'diffs'])

In [ ]:
M.active[0].keys()

In [7]:
root = os.environ['FOUNDATION_SAVE_DIR']
root

'/is/ei/fleeb/workspace/chome/trained_nets'

In [8]:
for r in M.active:
    n = r.name
    num = len(os.listdir(os.path.join(root, n)))
    if num > 100:
        print('{} : {}'.format(n, num))

3dshapes-dvae_0100-5966131-07_200109-133026 : 113
3dshapes-dvae_0101-5966132-01_200109-133045 : 114
3dshapes-dvae_0101-5966132-02_200109-133053 : 113
3dshapes-dvae_0102-5966133-07_200109-133118 : 113
3dshapes-dvae_0102-5966133-08_200109-133107 : 113
3dshapes-dvae_0122-5969012-01_200110-140610 : 113
3dshapes-dvae_0122-5969012-02_200110-140616 : 113
3dshapes-dwae_0094-5966044-04_200108-175514 : 113
3dshapes-dwae_0095-5966045-04_200108-175850 : 113
3dshapes-dwae_0096-5966047-04_200108-175913 : 113
3dshapes-dwae_0097-5966051-04_200108-181233 : 113
3dshapes-dwae_0100-5966131-05_200109-133027 : 113
3dshapes-dwae_0100-5966131-06_200109-133124 : 113
3dshapes-dwae_0101-5966132-00_200109-133044 : 113
3dshapes-dwae_0102-5966133-05_200109-133202 : 113
3dshapes-dwae_0102-5966133-06_200109-133106 : 113
3dshapes-dwae_0122-5969012-00_200110-140613 : 113
3dshapes-fdwae_0094-5966044-05_200108-175529 : 113
3dshapes-fdwae_0095-5966045-05_200108-175832 : 113
3dshapes-fdwae_0096-5966047-05_200108-175918 : 1

In [18]:
import json

In [19]:
json.loads('[10,2,3]')

[10, 2, 3]

In [ ]:
saveroot = os.environ['FOUNDATION_SAVE_DIR']
saveroot

In [ ]:
for name in os.listdir(saveroot):
    path = os.path.join(saveroot, name)
    if 'config.yml' in os.listdir(path):
        print(name,end=' ')
        with open(os.path.join(path, 'config.yml'), 'r') as f:
            data = yaml.load(f)
        break
#         if 'parents' in data:
#             del data['parents']
#             with open(os.path.join(path, 'config.yml'), 'w') as f:
#                 yaml.dump(data, f)
#             print('fixed')
#         else:
#             print()

In [ ]:
data['parents']